In [1]:
import keras
import tensorflow as tf

2025-04-17 20:04:13.928209: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-17 20:04:13.931748: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-17 20:04:13.939711: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744913053.952873    3381 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744913053.956506    3381 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744913053.967400    3381 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [15]:
from functools import partial

class MobileViT_V3XXS():

  def build_model(input_shape,include_top=True,num_classes=5,expansion_factor=2,patch_size=4,**kwargs):

    conv_block = partial(keras.layers.Conv2D,filters=16,kernel_size=3,strides=2, activation=keras.activations.swish,padding="same")
    


    inputs = keras.Input(shape=input_shape)




    x = conv_block( filters=16)(inputs) #  conv_block( x,filters=16)
    x = MobileViT_V3XXS.inverted_residual_block(
       x=x, expanded_channels=16 *expansion_factor, output_channels=16
    )

    # Downsampling with MV2 block.
    x = MobileViT_V3XXS.inverted_residual_block(
        x, expanded_channels=16 * expansion_factor, output_channels=24, strides=2
    )
    x = MobileViT_V3XXS.inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=24
    )
    x = MobileViT_V3XXS.inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=24
    )

    # First MV2 -> MobileViT block.
    x = MobileViT_V3XXS.inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=64, strides=2
    )
    x = MobileViT_V3XXS.mobilevit_block_v3(x=x,conv_block=conv_block,patch_size=patch_size, num_blocks=2, projection_dim=64)

    # Second MV2 -> MobileViT block.
    x = MobileViT_V3XXS.inverted_residual_block(
        x, expanded_channels=64 * expansion_factor, output_channels=80, strides=2
    )
    x = MobileViT_V3XXS.mobilevit_block_v3(x,conv_block=conv_block,patch_size=patch_size, num_blocks=4, projection_dim=80)

    # Third MV2 -> MobileViT block.
    x = MobileViT_V3XXS.inverted_residual_block(
        x, expanded_channels=80 * expansion_factor, output_channels=96, strides=2
    )
    x = MobileViT_V3XXS.mobilevit_block_v3(x,conv_block=conv_block,patch_size=patch_size, num_blocks=3, projection_dim=96)
    x = conv_block( filters=320, kernel_size=1, strides=1)(x)

    # Classification head.



    if include_top:
      x = keras.layers.GlobalAvgPool2D()(x)
      outputs = keras.layers.Dense(num_classes, activation="softmax")(x)
    else:
      outputs = x


    return keras.Model(inputs, outputs,name='ViT3XXS')





  def correct_pad(inputs, kernel_size):
      img_dim = 2 if keras.backend.image_data_format() == "channels_first" else 1
      input_size = inputs.shape[img_dim : (img_dim + 2)]
      if isinstance(kernel_size, int):
          kernel_size = (kernel_size, kernel_size)
      if input_size[0] is None:
          adjust = (1, 1)
      else:
          adjust = (1 - input_size[0] % 2, 1 - input_size[1] % 2)
      correct = (kernel_size[0] // 2, kernel_size[1] // 2)
      return (
          (correct[0] - adjust[0], correct[0]),
          (correct[1] - adjust[1], correct[1]),
      )


  # Reference: https://git.io/JKgtC


  def inverted_residual_block(x, expanded_channels, output_channels, strides=1):
      m = keras.layers.Conv2D(expanded_channels, 1, padding="same", use_bias=False)(x)
      m = keras.layers.BatchNormalization()(m)
      m = keras.activations.swish(m)

      if strides == 2:
          m = keras.layers.ZeroPadding2D(padding=MobileViT_V3XXS.correct_pad(m, 3))(m)
      m = keras.layers.DepthwiseConv2D(
          3, strides=strides, padding="same" if strides == 1 else "valid", use_bias=False
      )(m)
      m = keras.layers.BatchNormalization()(m)
      m = keras.activations.swish(m)

      m = keras.layers.Conv2D(output_channels, 1, padding="same", use_bias=False)(m)
      m = keras.layers.BatchNormalization()(m)

      if keras.ops.equal(x.shape[-1], output_channels) and strides == 1:
          return keras.layers.Add()([m, x])
      return m


  # Reference:
  # https://keras.io/examples/vision/image_classification_with_vision_transformer/


  def mlp(x, hidden_units, dropout_rate):
      for units in hidden_units:
          x = keras.layers.Dense(units, activation=keras.activations.swish)(x)
          x = keras.layers.Dropout(dropout_rate)(x)
      return x


  def transformer_block(x, transformer_layers, projection_dim, num_heads=2):
      for _ in range(transformer_layers):
          # Layer normalization 1.
          x1 = keras.layers.LayerNormalization(epsilon=1e-6)(x)
          # Create a multi-head attention layer.
          attention_output = keras.layers.MultiHeadAttention(
              num_heads=num_heads, key_dim=projection_dim, dropout=0.1
          )(x1, x1)
          # Skip connection 1.
          x2 = keras.layers.Add()([attention_output, x])
          # Layer normalization 2.
          x3 = keras.layers.LayerNormalization(epsilon=1e-6)(x2)
          # MLP.
          x3 = MobileViT_V3XXS.mlp(
              x3,
              hidden_units=[x.shape[-1] * 2, x.shape[-1]],
              dropout_rate=0.1,
          )
          # Skip connection 2.
          x = keras.layers.Add()([x3, x2])

      return x

  def mobilevit_block_v3(x,conv_block,patch_size,num_blocks, projection_dim, strides=1):
      local_features = keras.layers.DepthwiseConv2D(kernel_size=(3,3),strides=strides, activation=keras.activations.swish,padding="same")(x)
      local_features =  conv_block(
          filters=projection_dim, kernel_size=1, strides=strides
      )(local_features)

      skip2 = local_features


      # Unfold into patches and then pass through Transformers.
      num_patches = int((local_features.shape[1] * local_features.shape[2]) / patch_size)

      non_overlapping_patches = keras.layers.Reshape((patch_size, num_patches, projection_dim))(
          local_features
      )

      global_features = MobileViT_V3XXS.transformer_block(
          non_overlapping_patches, num_blocks, projection_dim
      )

      # Fold into conv-like feature-maps.
      folded_feature_map = keras.layers.Reshape((*local_features.shape[1:-1], projection_dim))(
          global_features
      )

      # Apply point-wise conv -> concatenate with the input features.
      folded_feature_map =  conv_block(
          filters=skip2.shape[-1], kernel_size=1, strides=strides
      )(folded_feature_map)

      local_global_features = keras.layers.Concatenate(axis=-1)([skip2, folded_feature_map])

      # Fuse the local and global features using a convoluion layer.
      local_global_features =  conv_block( filters=projection_dim, strides=strides)(local_global_features)

      return local_global_features+x


        


In [3]:
new_mobilevit = MobileViT_V3XXS.build_model((64,64,3),include_top=True,num_classes=10)


E0000 00:00:1744913058.858198    3381 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1744913058.879714    3381 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[sudo] password for bap: 
sudo: a password is required
^C


In [ ]:
from keras import layers,backend
# Values are from table 4.
patch_size = 4  # 2x2, for the Transformer blocks.
image_size = 64
expansion_factor = 2  # expansion factor for the MobileNetV2 blocks.


def conv_block(x, filters=16, kernel_size=3, strides=2):
    conv_layer = layers.Conv2D(
        filters,
        kernel_size,
        strides=strides,
        activation=keras.activations.swish,
        padding="same",
    )
    return conv_layer(x)


# Reference: https://github.com/keras-team/keras/blob/e3858739d178fe16a0c77ce7fab88b0be6dbbdc7/keras/applications/imagenet_utils.py#L413C17-L435


def correct_pad(inputs, kernel_size):
    img_dim = 2 if backend.image_data_format() == "channels_first" else 1
    input_size = inputs.shape[img_dim : (img_dim + 2)]
    if isinstance(kernel_size, int):
        kernel_size = (kernel_size, kernel_size)
    if input_size[0] is None:
        adjust = (1, 1)
    else:
        adjust = (1 - input_size[0] % 2, 1 - input_size[1] % 2)
    correct = (kernel_size[0] // 2, kernel_size[1] // 2)
    return (
        (correct[0] - adjust[0], correct[0]),
        (correct[1] - adjust[1], correct[1]),
    )


# Reference: https://git.io/JKgtC


def inverted_residual_block(x, expanded_channels, output_channels, strides=1):
    m = layers.Conv2D(expanded_channels, 1, padding="same", use_bias=False)(x)
    m = layers.BatchNormalization()(m)
    m = keras.activations.swish(m)

    if strides == 2:
        m = layers.ZeroPadding2D(padding=correct_pad(m, 3))(m)
    m = layers.DepthwiseConv2D(
        3, strides=strides, padding="same" if strides == 1 else "valid", use_bias=False
    )(m)
    m = layers.BatchNormalization()(m)
    m = keras.activations.swish(m)

    m = layers.Conv2D(output_channels, 1, padding="same", use_bias=False)(m)
    m = layers.BatchNormalization()(m)

    if keras.ops.equal(x.shape[-1], output_channels) and strides == 1:

        return layers.Add()([m, x])
    return m


# Reference:
# https://keras.io/examples/vision/image_classification_with_vision_transformer/


def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=keras.activations.swish)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


def transformer_block(x, transformer_layers, projection_dim, num_heads=2):
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, x])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(
            x3,
            hidden_units=[x.shape[-1] * 2, x.shape[-1]],
            dropout_rate=0.1,
        )
        # Skip connection 2.
        x = layers.Add()([x3, x2])

    return x

def mobilevit_block_v1(x, num_blocks, projection_dim, strides=1):
    # Local projection with convolutions.
    local_features = conv_block(x, filters=projection_dim, strides=strides)
    local_features = conv_block(
        local_features, filters=projection_dim, kernel_size=1, strides=strides
    )

    # Unfold into patches and then pass through Transformers.
    num_patches = int((local_features.shape[1] * local_features.shape[2]) / patch_size)
    non_overlapping_patches = layers.Reshape((patch_size, num_patches, projection_dim))(
        local_features
    )
    global_features = transformer_block(
        non_overlapping_patches, num_blocks, projection_dim
    )

    # Fold into conv-like feature-maps.
    folded_feature_map = layers.Reshape((*local_features.shape[1:-1], projection_dim))(
        global_features
    )

    # Apply point-wise conv -> concatenate with the input features.
    folded_feature_map = conv_block(
        folded_feature_map, filters=x.shape[-1], kernel_size=1, strides=strides
    )
    local_global_features = layers.Concatenate(axis=-1)([x, folded_feature_map])

    # Fuse the local and global features using a convoluion layer.
    local_global_features = conv_block(
        local_global_features, filters=projection_dim, strides=strides
    )

    return local_global_features


def create_mobilevitV1(num_classes=5):
    inputs = keras.Input((image_size, image_size, 3))
    x = layers.Rescaling(scale=1.0 / 255)(inputs)

    # Initial conv-stem -> MV2 block.
    x = conv_block(x, filters=16)
    x = inverted_residual_block(
        x, expanded_channels=16 * expansion_factor, output_channels=16
    )

    # Downsampling with MV2 block.
    x = inverted_residual_block(
        x, expanded_channels=16 * expansion_factor, output_channels=24, strides=2
    )
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=24
    )
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=24
    )

    # First MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=48, strides=2
    )
    x = mobilevit_block_v1(x, num_blocks=2, projection_dim=64)

    # Second MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=64 * expansion_factor, output_channels=64, strides=2
    )
    x = mobilevit_block_v1(x, num_blocks=4, projection_dim=80)

    # Third MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=80 * expansion_factor, output_channels=80, strides=2
    )
    x = mobilevit_block_v1(x, num_blocks=3, projection_dim=96)
    x = conv_block(x, filters=320, kernel_size=1, strides=1)

    # Classification head.
    x = layers.GlobalAvgPool2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(inputs, outputs)



In [5]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

test_images.shape

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


(10000, 32, 32, 3)

In [ ]:
ViT3 = MobileViT_V3XXS((64,64,3),include_top=True,num_classes=10,is_imagenet=True).get_model()

In [8]:
ViT3Complete= keras.Sequential([keras.Input(shape=(32,32,3)),
                                  keras.layers.Resizing(height=64,width=64),
                                  new_mobilevit])





In [ ]:
ViT1 = create_mobilevitV1(num_classes=10)
ViT1Complete = keras.Sequential([keras.Input(shape=(32,32,3)),
                                  keras.layers.Resizing(height=64,width=64),
                                  ViT1])

In [ ]:
early = keras.callbacks.EarlyStopping(patience=4,restore_best_weights=True)
ViT1Complete.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss='SparseCategoricalCrossentropy',metrics=['accuracy'])
ViT1Complete.fit(train_images,train_labels,validation_data=(test_images,test_labels),batch_size=128,epochs=100,callbacks=[early])

Epoch 1/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 173s 164ms/step - accuracy: 0.3793 - loss: 1.6495 - val_accuracy: 0.1000 - val_loss: 2.7317
Epoch 2/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 55s 45ms/step - accuracy: 0.6190 - loss: 1.0570 - val_accuracy: 0.1065 - val_loss: 5.4029
Epoch 3/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 45ms/step - accuracy: 0.7026 - loss: 0.8364 - val_accuracy: 0.2372 - val_loss: 2.7262
Epoch 4/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 17s 44ms/step - accuracy: 0.7557 - loss: 0.6819 - val_accuracy: 0.1088 - val_loss: 4.2970
Epoch 5/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 44ms/step - accuracy: 0.7983 - loss: 0.5717 - val_accuracy: 0.1930 - val_loss: 3.2019
Epoch 6/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step - accuracy: 0.8256 - loss: 0.4874 - val_accuracy: 0.3460 - val_loss: 2.3065
Epoch 7/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 17s 43ms/step - accuracy: 0.8612 - loss: 0.3945 - val_accuracy: 0.1325 - val_loss: 4.5710
Epoch 8/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - accuracy: 0.8791 - loss:

In [ ]:
early = keras.callbacks.EarlyStopping(patience=4,restore_best_weights=True)
ViT3Complete.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss='SparseCategoricalCrossentropy',metrics=['accuracy'])
ViT3Complete.fit(train_images,train_labels,validation_data=(test_images,test_labels),batch_size=128,epochs=100,callbacks=[early])

Epoch 1/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 165s 193ms/step - accuracy: 0.3554 - loss: 1.7163 - val_accuracy: 0.1000 - val_loss: 2.9361
Epoch 2/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 18s 45ms/step - accuracy: 0.5913 - loss: 1.1411 - val_accuracy: 0.5885 - val_loss: 1.1577
Epoch 3/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 44ms/step - accuracy: 0.6680 - loss: 0.9198 - val_accuracy: 0.6623 - val_loss: 0.9739
Epoch 4/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - accuracy: 0.7218 - loss: 0.7853 - val_accuracy: 0.6809 - val_loss: 0.9212
Epoch 5/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 44ms/step - accuracy: 0.7588 - loss: 0.6803 - val_accuracy: 0.6957 - val_loss: 0.8798
Epoch 6/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 18s 46ms/step - accuracy: 0.7849 - loss: 0.6087 - val_accuracy: 0.6964 - val_loss: 0.9420
Epoch 7/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 44ms/step - accuracy: 0.8103 - loss: 0.5403 - val_accuracy: 0.7272 - val_loss: 0.8256
Epoch 8/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - accuracy: 0.8269 - loss:

In [ ]:
acc_vit1=ViT1Complete.evaluate(test_images,test_labels)[1]
acc_vit3=ViT3Complete.evaluate(test_images,test_labels)[1]


print("ViT1 accuracy="+str(acc_vit1)+f'\nViT3 accuracy='+str(acc_vit3))

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.3505 - loss: 2.3187
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7456 - loss: 0.7831
ViT1 accuracy=0.34599998593330383
ViT3 accuracy=0.7445999979972839


In [ ]:
ViT3 = MobileViT_V3XXS((64,64,3),include_top=True,num_classes=10,is_imagenet=True).get_model()
ViT3.load_weights("./best_weights.weights.h5")
ViT3_best_trained= keras.Sequential([keras.Input(shape=(32,32,3)),
                                  keras.layers.Resizing(height=64,width=64),
                                  ViT3])

ViT3_best_trained.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss='SparseCategoricalCrossentropy',metrics=['accuracy'])
acc_vit3_best=ViT3_best_trained.evaluate(test_images,test_labels)[1]
print("best ViT3 weights accuracy="+str(acc_vit3_best))

313/313 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.8390 - loss: 0.4859
best ViT3 weights accuracy=0.8385999798774719
